In [ ]:
import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


rawname_ = 'S01_off_hold'
rawname_ = 'S01_on_hold'
rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'

rawname = rawname_ + '_resample_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)

In [ ]:
reconst_name = rawname_ + '_resample_raw.fif'
reconst_fname_full = os.path.join(data_dir,reconst_name)
raw = mne.io.read_raw_fif(reconst_fname_full, None)

In [ ]:
emgonly = raw.copy()

In [ ]:
help(emgonly.pick_channels)

In [ ]:
emgonly.load_data()

In [ ]:
#tp for tp in emgonly.get_channel_types() if tp == 'emg'

In [ ]:
chis = mne.pick_channels_regexp(emgonly.ch_names, 'EMG.*old')

In [ ]:
chis

In [ ]:
restr_names = np.array( emgonly.ch_names )[chis]
restr_names = restr_names.tolist()

#help(emgonly.pick_channels)

In [ ]:
emgonly.pick_channels(restr_names)

In [ ]:
chdata,times = emgonly[emgonly.ch_names]

In [ ]:
dat0 = chdata[0]

In [ ]:
help(emgonly.plot_psd)

In [ ]:
emgonly.ch_names

In [ ]:
emgonly[chname]

In [ ]:
for chname in emgonly.ch_names:
    chd, times = emgonly[chname]
    psd, freq = plt.psd( chd[0] , Fs= emgonly.info['sfreq'], label=chname)
plt.legend()

In [ ]:
help(emgonly.plot_psd)

In [ ]:
y = {}
for chname in emgonly.ch_names:
    y[chname] = 'eeg'
emgonly.set_channel_types(y)

In [ ]:
emgonly.plot_psd(color=['r']*4, fmax=40);

In [ ]:
help(emgonly.filter)

In [ ]:
emgonly.filter(l_freq=10, h_freq=None)

In [ ]:
for i in range(chdata.shape[0]):
    plt.hist( chdata[i], bins=100 , alpha=0.7, label=emgonly.ch_names[i])
plt.legend()

In [ ]:
for i in [1]:
    plt.hist( chdata[i], bins=100 , alpha=0.7, label=emgonly.ch_names[i])
plt.legend()

In [ ]:
rect = np.abs( emgonly.get_data() )

In [ ]:
sh = 3e-4
for i in range(rect.shape[0]):
    plt.plot( times, sh*i+ rect[i], alpha = 0.7, label=emgonly.ch_names[i] )
    
plt.legend()

In [ ]:
t0,t1 = emgonly.time_as_index([195,210])

In [ ]:
sh = 3e-4
for i in range(rect.shape[0]):
    plt.plot( times[t0:t1], sh*i+ rect[i][t0:t1], alpha = 0.7, label=emgonly.ch_names[i] )
    
plt.legend()

In [ ]:
maxtremfreq = 9
windowsz = int( emgonly.info['sfreq'] / maxtremfreq ) 
print( 'wind size is {} s = {} bins'.format(windowsz/emgonly.info['sfreq'], windowsz ))

# Smooth rectified

In [ ]:
sh = 6e-4
plt.figure(figsize=(14,4))
for i in range(rect.shape[0]):
    datcur = rect[i]
    datcur = np.convolve(datcur, np.ones(windowsz),  mode='same')
    ax = plt.subplot(1,2,1)
    ax.plot( times[t0:t1], sh*i+ datcur[t0:t1], alpha = 0.7, label=emgonly.ch_names[i] )
    ax.legend()
    
    ax = plt.subplot(1,2,2)
    ax.psd(datcur, Fs=emgonly.info['sfreq'], label=emgonly.ch_names[i])
    ax.set_xlim(0,20)
    
    ax.legend()

In [ ]:
b,a

In [ ]:
help( mne.filter.filter_data )

In [ ]:
sfreq = int( emgonly.info['sfreq'] )

# highpass smoothened rectification

In [ ]:
sh = 6e-4
plt.figure(figsize=(14,4))
for i in range(rect.shape[0]):
    datcur = rect[i]
    datcur = np.convolve(datcur,  np.ones(windowsz),  mode='same')
    datcur = mne.filter.filter_data(datcur, sfreq, 3,10 ,verbose=0)
    ax = plt.subplot(1,2,1)
    ax.plot( times[t0:t1], sh*i+ datcur[t0:t1], alpha = 0.7, label=emgonly.ch_names[i] )
    ax.legend()
    
    ax = plt.subplot(1,2,2)
    ax.psd(datcur, Fs=emgonly.info['sfreq'], label=emgonly.ch_names[i])
    ax.set_xlim(0,20)
    
    ax.legend()

In [ ]:
datcur.real

In [ ]:
datcur.imag

In [ ]:
phase

# extract amplitude and phase

In [ ]:
#help(plt.hist)

In [ ]:
axs_.shape

In [ ]:
import scipy.signal as sig

In [ ]:
%matplotlib qt
sh = 6e-4
nr = 4
nc = 2
fig,axs = plt.subplots(nrows=nr,ncols=nc, figsize=(16,14)) 
axs_ = axs.T.reshape(axs.size)
alpha = 0.5
for i in range(rect.shape[0]):
    datcur = rect[i]
    datcur = np.convolve(datcur,  np.ones(windowsz),  mode='same')
    datcur = mne.filter.filter_data(datcur, sfreq, 3,10 ,verbose=0)
    #datcur = mne.filter.filter_data(datcur, sfreq, 3,10 ,verbose=0)
    datcur = sig.hilbert(datcur)
    ang = np.angle(datcur)
    instphase = np.unwrap(ang)
    instfreq = (np.diff(instphase) / (2.0*np.pi) * sfreq)
    instampl = np.abs(datcur)
    
    ii = 0
    ax = axs[ii,0]; 
    ax.plot( times[t0:t1], sh*i+ datcur.real[t0:t1], alpha = alpha, label=emgonly.ch_names[i] )
    ax.set_title('Hilbert real')
    ax.axhline(y=sh*i, ls=':')
    ax.legend()
    
    ax = axs[ii,1]; 
    ax.hist( datcur.real, bins=100 , alpha=alpha, label=emgonly.ch_names[i])
    
    ii+=1
    ax = axs[ii,0]; 
    ax.plot( times[t0:t1], sh*i+ ang[t0:t1], alpha = alpha, label=emgonly.ch_names[i] )
    #ax.axhline(y=sh*i, ls=':')
    ax.set_title('Hilbert phase')
    
    ax = axs[ii,1]; 
    ax.hist( instphase, bins=100 , alpha=alpha, label=emgonly.ch_names[i])
    
    ii+=1
    ax = axs[ii,0]; 
    shfreq = 0
    #shfreq = 40
    ax.plot( times[t0:t1], shfreq*i+ instfreq[t0:t1], alpha = alpha, label=emgonly.ch_names[i] )
    ax.axhline(y=shfreq*i, ls=':')
    ax.set_title('Hilbert freq')
    ax.set_ylim(-10,10)
    
    ax = axs[ii,1]; ii+=1
    ax.hist( instfreq, bins=100 , range=(-10,10), alpha=alpha, label=emgonly.ch_names[i])
    #ax.set_xlim(-10,10)
    
    
    ax = axs[ii,0]; 
    ax.plot( times[t0:t1], sh*i+ instampl[t0:t1], alpha = alpha, label=emgonly.ch_names[i] )
    ax.axhline(y=sh*i, ls=':')
    ax.set_title('Hilbert abs')
    
    ax = axs[ii,1]; ii+=1
    ax.hist( instampl, bins=100 , alpha=alpha, label=emgonly.ch_names[i])
    
    ax.legend()


In [ ]:
for i in range(chdata.shape[0]):
    plt.hist( chdata[i], bins=100 , alpha=0.7, label=emgonly.ch_names[i])
plt.legend()

In [ ]:
# high passed filtered
rectconvraw = emgonly.copy()
#hilbraw.plot(duration=2)

rectconvraw.apply_function( np.abs)
rectconvraw.apply_function( lambda x: np.convolve(x,  np.ones(windowsz),  mode='same') )
rectconvraw.apply_function( lambda x: x / np.quantile(x,0.75) )
hilbraw = rectconvraw.copy()
hilbraw.filter(l_freq=2,h_freq=10)
hilbraw.apply_hilbert()

# smoothness of hilb_freq depends heavilly on the band we use for filtering hilbraw
hilb_amp = hilbraw.copy()
hilb_amp.apply_function(np.abs)

hilb_phase = hilbraw.copy()
hilb_phase.apply_function(np.angle)
#hilb_phase.apply_function( lambda x: x.real)

hilb_freq = hilb_phase.copy()
hilb_freq.apply_function(lambda x: (np.diff(np.unwrap(x.real),prepend=[0]) / (2.0*np.pi) * sfreq))

In [ ]:
rectconv_fname_full = os.path.join(data_dir, '{}_emg_rectconv.fif'.format(rawname_) )
rectconvraw.save(rectconv_fname_full, overwrite=1)

In [ ]:
d = {}
for chn in hilb_freq.ch_names:
    d[chn]='emg'
hilb_freq.set_channel_types(d)

In [ ]:
hilb_freq.info['chs']

In [ ]:
hilb_freq.info

In [ ]:
hilb_phase.get_data()

In [ ]:
phd.shape

In [ ]:
phd,tms = hilb_freq[:]
plt.plot(tms,phd[0].real)
plt.xlim(300,400)
plt.ylim(1,10)

In [ ]:
phd,tms = hilb_phase[:]
plt.plot(tms,phd[0].real)
plt.xlim(0,5)

In [ ]:
phd[0]

In [ ]:
phd,tms = hilb_amp[:]
plt.plot(tms,phd.T.real)
plt.xlim(300,None)
#plt.ylim(1,10)

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))
phd,tms = hilb_freq[:]
ax = plt.subplot(2,2,1)
ax.plot(tms,phd.T.real)
ax.set_xlim(300,370)
ax.set_ylim(0,10)

ax = plt.subplot(2,2,2)
ax.plot(tms,phd.T.real)
ax.set_xlim(180,220)
ax.set_ylim(0,10)


#plt.ylim(1,10)

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))
phd,tms = rectconvraw[:]
ax = plt.subplot(2,2,1)
ax.plot(tms,phd.T.real)
ax.set_xlim(300,370)

ax = plt.subplot(2,2,2)
ax.plot(tms,phd.T.real)
ax.set_xlim(180,220)

phd,tms = hilb_amp[:]
ax = plt.subplot(2,2,3)
ax.plot(tms,phd.T.real)
ax.set_xlim(300,370)

ax = plt.subplot(2,2,4)
ax.plot(tms,phd.T.real)
ax.set_xlim(180,220)


#plt.ylim(1,10)

In [ ]:
hilb_freq.plot(duration=2)

In [ ]:
hilb_phase.plot(duration=2)

In [ ]:
hilb_amp.plot(duration=2)

In [ ]:
import scipy.signal as sig

In [ ]:
sig.hilbert

In [ ]:
help(sig.butter)

In [ ]:
help(sig.hilbert)

In [ ]:
help(sig.hilbert2)

In [ ]:
sig.hilbert2(np.ones(20))